In [1]:
import numpy as np
import scipy
import scipy.sparse as sp
from scipy.sparse import eye, diags, block_diag, hstack
from scipy.sparse.csgraph import reverse_cuthill_mckee
from scipy.sparse.linalg import gmres
# from scipy.sparse.csgraph import symmetrix_degree_order
import importlib
import time
import NavierStokes_Periodic_Solver as NS_Per
import numpy as np
# from EnSF_Sparse_obs_dct import EnSF
from skimage.restoration import inpaint
import cv2
from sklearn import decomposition
import scipy.fft
import cvxpy as cp
# from scipy.fftpack import fft, dct, idct
from joblib import Parallel, delayed

In [2]:
importlib.reload(NS_Per)

<module 'NavierStokes_Periodic_Solver' from '/Users/huynh/Desktop/Python/EnSF/NavierStokes/Periodic/NavierStokes_Periodic/NavierStokes_Periodic_Solver.py'>

## Run on MATLAB first to generate 'Permutation_Indices_RefSol_Per40.mat', then run 'EnSF_DRLM_NS_Periodic' for reference solution

In [3]:
data = scipy.io.loadmat('Permutation_Indices_RefSol_Per40.mat')

# Convert MATLAB arrays to NumPy arrays and cast to float64.
perS = data['perS'].astype(np.float64)
# perA = data['perA'].astype(np.float64)

In [4]:
perS -= 1
# perA -= 1

In [5]:
# Assume the following helper functions are defined:
# -------------------------------
# Domain discretization and time setup
xa = 0 
xb = 1
ya = 0 
yb = 1 
T = 1

mu = 0.001
# n = 8; Nx = 32*n; Ny = Nx; Nt = 8*n;

theta = 5
Nx = 40
Ny = Nx
Nt = T*600

hx = (xb - xa) / Nx
hy = (yb - ya) / Ny
# Create grid points: MATLAB: x = xa:hx:xb, y = ya:hy:yb
x = np.arange(xa, xb + hx/2, hx)  # adding hx/2 ensures xb is included
y = np.arange(ya, yb + hy/2, hy)
xmid = NS_Per.avg(x)
ymid = NS_Per.avg(y)
dt = T / Nt
TT = np.arange(0, T + dt/2, dt)

alpha_BE = 1 / dt   # alpha*u - mu*Delta(u) + grad(p) = f
opt_UgradU = 1   # 1: original, 2: MIT (not good)
opt = 2
# -------------------------------
# Construct matrices A and B

# Sizes:
sU = (Nx) * Ny       # for U-velocity unknowns
sV = sU       # for V-velocity unknowns
sP = sU             # for pressure


# --- Build matrix A ---
A0  = NS_Per.DiscreteLaplace(Nx,hx)
B0  = NS_Per.DiscreteLaplace(Ny,hy)

A_u = alpha_BE * sp.eye(sU) - mu * (sp.kron(sp.eye(Ny), A0) + sp.kron(B0, sp.eye(Nx)))
A_v = A_u.copy()

A_BE = block_diag((A_u, A_v), format='csr')

# # --- Construct matrix B ---

A1 = NS_Per.DiscreteGrad(Nx,hx)         # P_x = A1*P
B1 = NS_Per.DiscreteGrad(Ny,hy)         # P_y = P*B1'

B2 = sp.kron(sp.eye(Ny), A1.T)
B3 = sp.kron(B1.T, sp.eye(Nx))
B = sp.hstack([B2, B3], format='csr')

B = B[1:, :]
Bt = B.transpose().tocsr()

# # --- Prepare matrices for the pressure correction ---
dA_BE = A_BE.diagonal()
D_BE = diags(dA_BE, 0, shape=A_BE.shape, format='csr')
E_BE = D_BE - A_BE
Di_BE = diags(1.0 / dA_BE, 0, shape=A_BE.shape, format='csr')
S_BE = B.dot(Di_BE.dot(Bt))
# # perS = reverse_cuthill_mckee(S)
# # S_perm = S[perS, :][:, perS].toarray()

rowsS, colsS = np.meshgrid(perS, perS)
S_perm = S_BE[rowsS, colsS].toarray()
SS_BE = np.linalg.cholesky(S_perm).T
SS_BEt = SS_BE.T
DiE_BE = Di_BE.dot(E_BE)
BDiE_BE = B.dot(DiE_BE)
DiB_BEt = Di_BE.dot(Bt)

## Create mesh
Yu, Xu = np.meshgrid(ymid, x[0:-1], indexing='xy')

Yv, Xv = np.meshgrid(y[0:-1], xmid, indexing='xy')

Yp, Xp = np.meshgrid(ymid, xmid, indexing='xy')

# Initialize velocity fields using your exact solution functions.
U0 = NS_Per.u_init(Xu, Yu, opt)  # dimensions should match (len(x[1:-1]) x len(ymid))
V0 = NS_Per.v_init(Xv, Yv, opt)

# Initialize pressure-related quantities.
q = 1

q_batch = np.full((2,), q)
# qq = np.zeros(Nt+1)
# qq[0] = q

# egy = np.zeros(Nt + 1)
# egy_theta = egy.copy()
# egy[0] = 0.5 * hx * hy * (NS_Per.inner(U0, U0) + NS_Per.inner(V0, V0))
# egy_theta[0] = egy[0]+theta*(q**2-1)

In [6]:
perSBE_new = perS.astype(int)

perSBE_new = np.squeeze(perSBE_new, axis=0)

In [7]:
data = scipy.io.loadmat('Permutation_Indices_RefSol_BDF2Per40.mat')

# Convert MATLAB arrays to NumPy arrays and cast to float64.
perBDFS = data['perS'].astype(np.float64)

In [8]:
perBDFS -= 1

In [9]:
# Assume the following helper functions are defined:
# -------------------------------
# Domain discretization and time setup
xa = 0 
xb = 1
ya = 0 
yb = 1 
T = 1

mu = 0.001
# n = 8; Nx = 32*n; Ny = Nx; Nt = 8*n;

theta = 5
Nx = 40
Ny = Nx
Nt = T*600

hx = (xb - xa) / Nx
hy = (yb - ya) / Ny
# Create grid points: MATLAB: x = xa:hx:xb, y = ya:hy:yb
x = np.arange(xa, xb + hx/2, hx)  # adding hx/2 ensures xb is included
y = np.arange(ya, yb + hy/2, hy)
xmid = NS_Per.avg(x)
ymid = NS_Per.avg(y)
dt = T / Nt
TT = np.arange(0, T + dt/2, dt)

alpha = 1.5 / dt   # alpha*u - mu*Delta(u) + grad(p) = f
opt_UgradU = 1   # 1: original, 2: MIT (not good)
opt = 2
# -------------------------------
# Construct matrices A and B

# Sizes:
sU = (Nx) * Ny       # for U-velocity unknowns
sV = sU       # for V-velocity unknowns
sP = sU             # for pressure


# --- Build matrix A ---
A0  = NS_Per.DiscreteLaplace(Nx,hx)
B0  = NS_Per.DiscreteLaplace(Ny,hy)

A_u = alpha * sp.eye(sU) - mu * (sp.kron(sp.eye(Ny), A0) + sp.kron(B0, sp.eye(Nx)))
A_v = A_u.copy()

A = block_diag((A_u, A_v), format='csr')

# # --- Construct matrix B ---

A1 = NS_Per.DiscreteGrad(Nx,hx)         # P_x = A1*P
B1 = NS_Per.DiscreteGrad(Ny,hy)         # P_y = P*B1'

B2 = sp.kron(sp.eye(Ny), A1.T)
B3 = sp.kron(B1.T, sp.eye(Nx))
B = sp.hstack([B2, B3], format='csr')

B = B[1:, :]
Bt = B.transpose().tocsr()

# # --- Prepare matrices for the pressure correction ---
dA = A.diagonal()
D = diags(dA, 0, shape=A.shape, format='csr')
E = D - A
Di = diags(1.0 / dA, 0, shape=A.shape, format='csr')
S = B.dot(Di.dot(Bt))
# # perS = reverse_cuthill_mckee(S)
# # S_perm = S[perS, :][:, perS].toarray()

rowsS, colsS = np.meshgrid(perBDFS, perBDFS)
S_perm = S[rowsS, colsS].toarray()
SS = np.linalg.cholesky(S_perm).T
SSt = SS.T
DiE = Di.dot(E)
BDiE = B.dot(DiE)
DiBt = Di.dot(Bt)

## Create mesh
Yu, Xu = np.meshgrid(ymid, x[0:-1], indexing='xy')

Yv, Xv = np.meshgrid(y[0:-1], xmid, indexing='xy')

Yp, Xp = np.meshgrid(ymid, xmid, indexing='xy')

# Initialize velocity fields using your exact solution functions.
U0 = NS_Per.u_init(Xu, Yu, opt)  # dimensions should match (len(x[1:-1]) x len(ymid))
V0 = NS_Per.v_init(Xv, Yv, opt)

# Initialize pressure-related quantities.
q = 1

# q_batch = np.full((2,), q)
# qq = np.zeros(Nt+1)
# qq[0] = q

# egy = np.zeros(Nt + 1)
# egy_theta = egy.copy()
# egy[0] = 0.5 * hx * hy * (NS_Per.inner(U0, U0) + NS_Per.inner(V0, V0))
# egy_theta[0] = egy[0]+theta*(q**2-1)

In [10]:
perBDFS_new = perBDFS.astype(int)

perBDFS_new = np.squeeze(perBDFS_new, axis=0)

In [13]:
mU0, nU0 = U0.shape
mV0, nV0 = V0.shape
mP0, nP0 = Xp.shape

In [23]:
All_U = np.zeros((Nt+1, mU0*nU0))
All_V = np.zeros((Nt+1, mV0*nV0))
All_P = np.zeros((Nt+1, mP0*nP0))
All_q = np.zeros((Nt+1, 1))

In [25]:
All_U[0, :] += np.reshape(U0, mU0*nU0, order='F')
All_V[0, :] += np.reshape(V0, mV0*nV0, order='F')
All_q[0, :] += 1

In [27]:
Size_U = mU0*nU0
Size_V = mV0*nV0
Size_P = mP0*nP0

In [65]:
## 10% arctangent observation

#### U-index
# idxU_obs = np.random.permutation(Size_U)[:int(0.1*Size_U)]
idxU_obs = np.random.permutation(Size_U)[:int(0.07*Size_U)]
idxU_unobs = np.setdiff1d(np.arange(Size_U), idxU_obs)

#### V-index
# idxV_obs = np.random.permutation(Size_V)[:int(0.1*Size_V)]+Size_U
idxV_obs = np.random.permutation(Size_V)[:int(0.07*Size_V)]+Size_U
idxV_unobs = np.setdiff1d(np.arange(Size_V)+Size_U, idxV_obs)

#### P-index
# idxP_obs = np.random.permutation(Size_P)[:int(0.1*Size_P)]+Size_U+Size_V
idxP_obs = np.random.permutation(Size_P)[:int(0.07*Size_P)]+Size_U+Size_V
idxP_unobs = np.setdiff1d(np.arange(Size_P)+Size_V+Size_U, idxP_obs)

idx_q = np.array(Size_U+Size_V+Size_P)


num_obs= idxU_obs.size+idxV_obs.size+idxP_obs.size+1

# Combine and sort the selected indices
idx_obs = np.sort(np.concatenate([idxU_obs, idxV_obs, idxP_obs, idx_q.reshape(1)]))

idx_unobs = np.sort(np.concatenate([idxU_unobs, idxV_unobs, idxP_unobs]))

(337,)
(4464,)


In [67]:
idx_Uobs_rows = idxU_obs // (Nx)
idx_Uobs_cols = idxU_obs % (Ny)

idx_Vobs_rows = (idxV_obs-Size_U) // (Nx)
idx_Vobs_cols = (idxV_obs-Size_U) % (Ny)

idx_Pobs_rows = (idxP_obs-Size_U-Size_V) // (Nx)
idx_Pobs_cols = (idxP_obs-Size_U-Size_V) % (Ny)

In [69]:
def cond_alpha(t):
    # conditional information
    # alpha_t(0) = 1
    # alpha_t(1) = esp_alpha \approx 0
    return 1 - (1-eps_alpha)*t


def cond_sigma_sq(t):
    # conditional sigma^2
    # sigma2_t(0) = 0
    # sigma2_t(1) = 1
    # sigma(t) = t
    return t

def cond_sigma_sq(t):
    # conditional sigma^2
    # sigma2_t(0) = 0
    # sigma2_t(1) = 1
    # sigma(t) = t
    return t

def f(t):
    # f=d_(log_alpha)/dt
    alpha_t = cond_alpha(t)
    f_t = -(1-eps_alpha) / alpha_t
    return f_t

def g_sq(t):
    # g = d(sigma_t^2)/dt -2f sigma_t^2
    d_sigma_sq_dt = 1
    g2 = d_sigma_sq_dt - 2*f(t)*cond_sigma_sq(t)
    return g2

def g(t):
    return np.sqrt(g_sq(t))

def reverse_SDE(obs, obs_sigma, x0, time_steps, C, score_likelihood_EnSF=None, drift_fun=f, \
                diffuse_fun=g, alpha_fun=cond_alpha, sigma2_fun=cond_sigma_sq, save_path=False):
    # x_T: sample from standard Gaussian
    # x_0: target distribution to sample from

    # reverse SDE sampling process
    # N1 = x_T.shape[0]
    # N2 = x0.shape[0]
    # d = x_T.shape[1]

    # Generate the time mesh
    dt = 1.0/time_steps

    # Initialization
    xt = np.random.randn(x0.shape[0], x0.shape[1])
    t = 1.0
    
    path_all = []
    t_vec = []
    
    # define storage
    if save_path:
        path_all.append(xt.copy())
        t_vec.append(t)
    
    # forward Euler sampling
    for i in range(time_steps):
        # prior score evaluation
        alpha_t = alpha_fun(t)
        sigma2_t = sigma2_fun(t)


        # Evaluate the diffusion term
        diffuse = diffuse_fun(t)

        # Evaluate the drift term
        # drift = drift_fun(t)*xt - diffuse**2 * score_eval

        # Update
        if score_likelihood is not None:
#             zt = score_likelihood(xt, t)
#             print(zt.size())
            xt += -dt*(drift_fun(t)*xt+diffuse**2*((xt-alpha_t*x0)/sigma2_t)-\
                       diffuse**2*score_likelihood_EnSF(xt, obs, obs_sigma, t, C))\
                       +np.sqrt(dt)*diffuse*np.random.randn(*xt.shape)
    
        else:
            xt += -dt*(drift_fun(t)*xt+diffuse**2*((xt-alpha_t*x0)/sigma2_t))\
                    +np.sqrt(dt)*diffuse*np.random.randn(*xt.shape)
        
#         xt = xt.to(torch.float64)
        # Store the state in the path
        if save_path:
            path_all.append(xt.copy())
            t_vec.append(t)

        # update time
        t = t - dt

    if save_path:
        return path_all, t_vec
    else:
        return xt

In [71]:
def inpaint_task(i, masked_img, img_mask):
    return inpaint.inpaint_biharmonic(masked_img[i], 1.0 - img_mask)

In [73]:
def process_image(i, cv2_raw_img, cv2_mask):
    return cv2.inpaint(cv2_raw_img[i], cv2_mask, 2, cv2.INPAINT_NS)/255

In [75]:
data1 = scipy.io.loadmat('TestRefSol_BDF2_Periodic_v2.mat')
U_Ref = data1['U_Py']
V_Ref = data1['V_Py']
P_Ref = data1['P_Py']
q_Ref = data1['q_Py']

In [77]:
# ntEnSF = 50
ntEnSF = 100
t0 = 0
filtering_steps = ntEnSF
timeTrue = np.linspace(0, 1, Nt+1)
tEnSF = np.linspace(0, 1, filtering_steps+1)
indices_time = np.searchsorted(timeTrue, tEnSF, side='left')


state_ref = np.concatenate((U_Ref, V_Ref, P_Ref, q_Ref), axis=1)   

state_timeextract = state_ref[indices_time, :].copy()

state_EnSF = state_timeextract[:, idx_obs].copy()

dtEnSF = (T - t0) / ntEnSF
obs_sigma = 0.1

eps_alpha = 0.05

# ensemble size
ensemble_size = 100
ensemble_true = 1
# forward Euler step
euler_steps = 500
def g_tau(t):
    return 1-t

U0_state = 2*np.random.randn(ensemble_size, Size_U)
V0_state = 2*np.random.randn(ensemble_size, Size_V)
P0_state = 2*np.random.randn(ensemble_size, Size_P)

UV_state = np.concatenate((U0_state, V0_state, P0_state, np.full((ensemble_size, 1), 1)), axis=1)

n_dim = Size_U+Size_V+Size_P+1
rmse_all = []
obs_save = []
est_save = np.zeros((filtering_steps+1, n_dim))
est_save[[0], :] += np.mean(UV_state, axis=0)

(601, 4801)


In [79]:
## Noise omega_1
SDE_Sigma_U = 0.001
SDE_Sigma_V = 0.001
SDE_Sigma_P = 0.001

## Noise omega_2
# SDE_Sigma_U = 0.1
# SDE_Sigma_V = 0.1
# SDE_Sigma_P = 0.1

In [81]:
data1 = scipy.io.loadmat('Permutation_Indices_EnSF_Per40_T100.mat')
# data1 = scipy.io.loadmat('Permutation_Indices_EnSF_Per40.mat')
# Convert MATLAB arrays to NumPy arrays and cast to float64.
perS_EnSF = data1['perS'].astype(np.float64)

In [83]:
perS_EnSF -= 1

In [85]:
# Assume the following helper functions are defined:
# -------------------------------
# Domain discretization and time setup
xa = 0 
xb = 1
ya = 0 
yb = 1 
T = 1

mu = 0.001
# n = 8; Nx = 32*n; Ny = Nx; Nt = 8*n;

theta = 5
Nx = 40
Ny = Nx
# Nt = T*50

hx = (xb - xa) / Nx
hy = (yb - ya) / Ny
# Create grid points: MATLAB: x = xa:hx:xb, y = ya:hy:yb
x = np.arange(xa, xb + hx/2, hx)  # adding hx/2 ensures xb is included
y = np.arange(ya, yb + hy/2, hy)
xmid = NS_Per.avg(x)
ymid = NS_Per.avg(y)
# dt = T / Nt
TTEnSF = np.arange(0, T + dtEnSF/2, dtEnSF)

alpha_EnSFBE = 1 / dtEnSF   # alpha*u - mu*Delta(u) + grad(p) = f
opt_UgradU = 1   # 1: original, 2: MIT (not good)
opt = 2
# -------------------------------
# Construct matrices A and B

# Sizes:
sU = (Nx) * Ny       # for U-velocity unknowns
sV = sU       # for V-velocity unknowns
sP = sU             # for pressure


# --- Build matrix A ---
A0  = NS_Per.DiscreteLaplace(Nx,hx)
B0  = NS_Per.DiscreteLaplace(Ny,hy)

A_u = alpha_EnSFBE * sp.eye(sU) - mu * (sp.kron(sp.eye(Ny), A0) + sp.kron(B0, sp.eye(Nx)))
A_v = A_u.copy()

A_EnSFBE = block_diag((A_u, A_v), format='csr')

# # --- Construct matrix B ---

A1 = NS_Per.DiscreteGrad(Nx,hx)         # P_x = A1*P
B1 = NS_Per.DiscreteGrad(Ny,hy)         # P_y = P*B1'

B2 = sp.kron(sp.eye(Ny), A1.T)
B3 = sp.kron(B1.T, sp.eye(Nx))
B = sp.hstack([B2, B3], format='csr')

B = B[1:, :]
Bt = B.transpose().tocsr()

# # --- Prepare matrices for the pressure correction ---
dA_EnSFBE = A_EnSFBE.diagonal()
D_EnSFBE = diags(dA_EnSFBE, 0, shape=A_EnSFBE.shape, format='csr')
E_EnSFBE = D_EnSFBE - A_EnSFBE
Di_EnSFBE = diags(1.0 / dA_EnSFBE, 0, shape=A_EnSFBE.shape, format='csr')
S_EnSFBE = B.dot(Di_EnSFBE.dot(Bt))
# # perS = reverse_cuthill_mckee(S)
# # S_perm = S[perS, :][:, perS].toarray()

rowsS, colsS = np.meshgrid(perS_EnSF, perS_EnSF)
S_perm = S_EnSFBE[rowsS, colsS].toarray()
SS_EnSFBE = np.linalg.cholesky(S_perm).T
SS_EnSFBEt = SS_EnSFBE.T
DiE_EnSFBE = Di_EnSFBE.dot(E_EnSFBE)
BDiE_EnSFBE = B.dot(DiE_EnSFBE)
DiB_EnSFBEt = Di_EnSFBE.dot(Bt)

## Create mesh
Yu, Xu = np.meshgrid(ymid, x[0:-1], indexing='xy')

Yv, Xv = np.meshgrid(y[0:-1], xmid, indexing='xy')

Yp, Xp = np.meshgrid(ymid, xmid, indexing='xy')


# Initialize pressure-related quantities.
q = 1

q_batch = np.full((ensemble_size,), q)
# qq = np.zeros(Nt+1)
# qq[0] = q

# egy = np.zeros(Nt + 1)
# egy_theta = egy.copy()
# egy[0] = 0.5 * hx * hy * (NS_Per.inner(U0, U0) + NS_Per.inner(V0, V0))
# egy_theta[0] = egy[0]+theta*(q**2-1)

In [87]:
perS_EnSFnew = perS_EnSF.astype(int)

perS_EnSFnew = np.squeeze(perS_EnSFnew, axis=0)

In [89]:
def score_likelihood(xt, obs, obs_sigma, t, C):
        # obs: (d)
        # xt: (ensemble, d)
        score_x = -(np.arctan(xt) - obs)/obs_sigma**2 * (1./(1+xt**2))
        tau = g_tau(t)
        return tau*score_x/C

In [91]:
def score_likelihood_unbos(xt, obs, obs_sigma, t, C):
        # obs: (d)
        # xt: (ensemble, d)
        score_x = -(xt - obs)/obs_sigma**2
        tau = g_tau(t)
        return tau*score_x/C

In [93]:
for i in range(filtering_steps):
    print(f'step={i}:')
    t1 = time.time()    
    
#     obs = state_EnSF[[i+1], :].copy()
    state_scale = state_EnSF[[i+1], :].copy()
    
    indob_scale0 = np.nonzero(((-1e-1 <= state_scale) & (state_scale < -1e-2)) |
                              ((1e-2 <= state_scale) & (state_scale < 1e-1)))[1]
    
    indob_scale1 = np.nonzero(((-1e-2 <= state_scale) & (state_scale < -1e-3)) |
                              ((1e-3 <= state_scale) & (state_scale < 1e-2)))[1]
    
    indob_scale2 = np.nonzero(((-1e-3 <= state_scale) & (state_scale < -1e-4)) |
                              ((1e-4 <= state_scale) & (state_scale < 1e-3)))[1]

    indob_scale3 = np.nonzero(((-1e-4 <= state_scale) & (state_scale < -1e-5)) |
                              ((1e-5 <= state_scale) & (state_scale < 1e-4)))[1]

    indob_scale4 = np.nonzero(((-1e-5 <= state_scale) & (state_scale < -1e-6)) |
                              ((1e-6 <= state_scale) & (state_scale < 1e-5)))[1]
    indob_scale5 = np.nonzero(((-1e-6 <= state_scale) & (state_scale < -1e-7)) |
                              ((1e-7 <= state_scale) & (state_scale < 1e-6)))[1]
    indob_scale6 = np.nonzero(((-1e-7 <= state_scale) & (state_scale < -1e-8)) |
                              ((1e-8 <= state_scale) & (state_scale < 1e-7)))[1]
    indob_scale7 = np.nonzero(((-1e-8 <= state_scale) & (state_scale < -1e-9)) |
                              ((1e-9 <= state_scale) & (state_scale < 1e-8)))[1]
    indob_scale8 = np.nonzero(((-1e-9 <= state_scale) & (state_scale < -1e-10)) |
                              ((1e-10 <= state_scale) & (state_scale < 1e-9)))[1]

    indob_scale9 = np.nonzero(((-1e-10 <= state_scale) & (state_scale < -1e-11)) |
                              ((1e-11 <= state_scale) & (state_scale < 1e-10)))[1]
    
    indob_scale10 = np.nonzero(((-1e-12 <= state_scale) & (state_scale < -1e-13)) |
                              ((1e-13 <= state_scale) & (state_scale < 1e-12)))[1]

    indob_scale11 = np.nonzero(((-1e-13 <= state_scale) & (state_scale < -1e-14)) |
                              ((1e-14 <= state_scale) & (state_scale < 1e-13)))[1]
    
    indob_scale12 = np.nonzero(((-1e-14 <= state_scale) & (state_scale < -1e-15)) |
                              ((1e-15 <= state_scale) & (state_scale < 1e-14)))[1]
    
    indob_scale13 = np.nonzero(((-1e-15 <= state_scale) & (state_scale < -1e-16)) |
                              ((1e-16 <= state_scale) & (state_scale < 1e-15)))[1]

    indob_scale14 = np.nonzero(((-1e-16 <= state_scale) & (state_scale < -1e-17)) |
                              ((1e-17 <= state_scale) & (state_scale < 1e-16)))[1]
    indob_scale15 = np.nonzero(((-1e-17 <= state_scale) & (state_scale < 0)) |
                              ((0 <= state_scale) & (state_scale < 1e-17)))[1]
    
    state_scale[:, indob_scale0] *= 1e1
    state_scale[:, indob_scale1] *= 1e2
    state_scale[:, indob_scale2] *= 1e3
    state_scale[:, indob_scale3] *= 1e4
    state_scale[:, indob_scale4] *= 1e5
    state_scale[:, indob_scale5] *= 1e6
    state_scale[:, indob_scale6] *= 1e7
    state_scale[:, indob_scale7] *= 1e8
    state_scale[:, indob_scale8] *= 1e9
    state_scale[:, indob_scale9] *= 1e10
    state_scale[:, indob_scale10] *= 1e12
    state_scale[:, indob_scale11] *= 1e13
    state_scale[:, indob_scale12] *= 1e14
    state_scale[:, indob_scale13] *= 1e15
    state_scale[:, indob_scale14] *= 1e16
    state_scale[:, indob_scale15] *= 1e17
    
    obs = np.arctan(state_scale.copy())
    obs += np.random.randn(*state_EnSF[[i+1], :].shape) * obs_sigma
       
    U_stack = (U0_state.T).copy()
    V_stack = (V0_state.T).copy()
    
    U_stack = U_stack.reshape(mU0, nU0, ensemble_size, order = 'F')
    V_stack = V_stack.reshape(mV0, nV0, ensemble_size, order = 'F')
    
    U_new, V_new, P_new, q_new, egy_new, egy_theta_new,_ = \
            NS_Per.NS_BE_1step_Periodic_Vectorized(hx, hy, dtEnSF, TTEnSF[i+1], U_stack, V_stack, q_batch, Xu, Yu, Xv, Yv,\
                                                   mu, theta, opt, opt_UgradU, DiE_EnSFBE, BDiE_EnSFBE, DiB_EnSFBEt,\
                                                   Di_EnSFBE, B, Bt, perS_EnSFnew, SS_EnSFBE, SS_EnSFBEt, Nx, Ny, sU,\
                                                   alpha_EnSFBE, A1, B1)
    
    U_new_reshape = U_new.reshape(mU0*nU0, ensemble_size, order ='F')  
    V_new_reshape = V_new.reshape(mV0*nV0, ensemble_size, order ='F')
    P_new_reshape = P_new.reshape(mP0*nP0, ensemble_size, order ='F')
    q_new_reshape = q_new.reshape(1, ensemble_size)
    
    # q_batch = q_new.copy()
    x_state = np.concatenate((U_new_reshape, V_new_reshape, P_new_reshape, q_new_reshape))
    
    noiseU = np.sqrt(dtEnSF) * SDE_Sigma_U * np.random.randn(*U_new_reshape.shape)
    noiseV = np.sqrt(dtEnSF) * SDE_Sigma_V * np.random.randn(*V_new_reshape.shape)
    noiseP = np.sqrt(dtEnSF) * SDE_Sigma_P * np.random.randn(*P_new_reshape.shape)

    noise = np.concatenate((noiseU, noiseV, noiseP, np.zeros((1, ensemble_size))))
    
    x_state += noise
    x_state = x_state.T
    
    x0_EnSF = x_state[:, idx_obs].copy()

    for l in range(5):      
        indx_scale0 = np.argwhere(((-1e-1<=x0_EnSF) & (x0_EnSF<-1e-2)) | ((1e-2<=x0_EnSF) & (x0_EnSF<1e-1)))
        indx_scale1 = np.argwhere(((-1e-2<=x0_EnSF) & (x0_EnSF<-1e-3)) | ((1e-3<=x0_EnSF) & (x0_EnSF<1e-2)))
        indx_scale2 = np.argwhere(((-1e-3<=x0_EnSF) & (x0_EnSF<-1e-4)) | ((1e-4<=x0_EnSF) & (x0_EnSF<1e-3)))
        indx_scale3 = np.argwhere(((-1e-4<=x0_EnSF) & (x0_EnSF<-1e-5)) | ((1e-5<=x0_EnSF) & (x0_EnSF<1e-4)))
        indx_scale4 = np.argwhere(((-1e-5<=x0_EnSF) & (x0_EnSF<-1e-6)) | ((1e-6<=x0_EnSF) & (x0_EnSF<1e-5)))
        indx_scale5 = np.argwhere(((-1e-6<=x0_EnSF) & (x0_EnSF<-1e-7)) | ((1e-7<=x0_EnSF) & (x0_EnSF<1e-6)))
        indx_scale6 = np.argwhere(((-1e-7<=x0_EnSF) & (x0_EnSF<-1e-8)) | ((1e-8<=x0_EnSF) & (x0_EnSF<1e-7)))
        indx_scale7 = np.argwhere(((-1e-8<=x0_EnSF) & (x0_EnSF<-1e-9)) | ((1e-9<=x0_EnSF) & (x0_EnSF<1e-8)))
        indx_scale8 = np.argwhere(((-1e-9<=x0_EnSF) & (x0_EnSF<-1e-10)) | ((1e-10<=x0_EnSF) & (x0_EnSF<1e-9)))
        indx_scale9 = np.argwhere(((-1e-10<=x0_EnSF) & (x0_EnSF<-1e-11)) | ((1e-11<=x0_EnSF) & (x0_EnSF<1e-10)))
        
        indx_scale10 = np.argwhere(((-1e-12<=x0_EnSF) & (x0_EnSF<-1e-13)) | ((1e-13<=x0_EnSF) & (x0_EnSF<1e-12)))
        indx_scale11 = np.argwhere(((-1e-13<=x0_EnSF) & (x0_EnSF<-1e-14)) | ((1e-14<=x0_EnSF) & (x0_EnSF<1e-13)))
        indx_scale12 = np.argwhere(((-1e-14<=x0_EnSF) & (x0_EnSF<-1e-15)) | ((1e-15<=x0_EnSF) & (x0_EnSF<1e-14)))
        indx_scale13 = np.argwhere(((-1e-15<=x0_EnSF) & (x0_EnSF<-1e-16)) | ((1e-16<=x0_EnSF) & (x0_EnSF<1e-15)))
        indx_scale14 = np.argwhere(((-1e-16<=x0_EnSF) & (x0_EnSF<-1e-17)) | ((1e-17<=x0_EnSF) & (x0_EnSF<1e-16)))
        indx_scale15 = np.argwhere(((-1e-17<=x0_EnSF) & (x0_EnSF<0)) | ((0<=x0_EnSF) & (x0_EnSF<1e-17)))
        
        x0_EnSF[indx_scale0[:, 0], indx_scale0[:, 1]] *= 1e1
        x0_EnSF[indx_scale1[:, 0], indx_scale1[:, 1]] *= 1e2
        x0_EnSF[indx_scale2[:, 0], indx_scale2[:, 1]] *= 1e3
        x0_EnSF[indx_scale3[:, 0], indx_scale3[:, 1]] *= 1e4
        x0_EnSF[indx_scale4[:, 0], indx_scale4[:, 1]] *= 1e5
        x0_EnSF[indx_scale5[:, 0], indx_scale5[:, 1]] *= 1e6
        x0_EnSF[indx_scale6[:, 0], indx_scale6[:, 1]] *= 1e7
        x0_EnSF[indx_scale7[:, 0], indx_scale7[:, 1]] *= 1e8
        x0_EnSF[indx_scale8[:, 0], indx_scale8[:, 1]] *= 1e9
        x0_EnSF[indx_scale9[:, 0], indx_scale9[:, 1]] *= 1e10
        x0_EnSF[indx_scale10[:, 0], indx_scale10[:, 1]] *= 1e12
        x0_EnSF[indx_scale11[:, 0], indx_scale11[:, 1]] *= 1e13
        x0_EnSF[indx_scale12[:, 0], indx_scale12[:, 1]] *= 1e14
        x0_EnSF[indx_scale13[:, 0], indx_scale13[:, 1]] *= 1e15
        x0_EnSF[indx_scale14[:, 0], indx_scale14[:, 1]] *= 1e16
        x0_EnSF[indx_scale15[:, 0], indx_scale15[:, 1]] *= 1e17

        sln_bar = reverse_SDE(obs, obs_sigma, x0=x0_EnSF.copy(), time_steps=euler_steps, \
                              C=1, score_likelihood_EnSF=score_likelihood)

        ## v1a
        sln_bar[:, indob_scale0] /= 1e1
        sln_bar[:, indob_scale1] /= 1e2
        sln_bar[:, indob_scale2] /= 1e3
        sln_bar[:, indob_scale3] /= 1e4
        sln_bar[:, indob_scale4] /= 1e5
        sln_bar[:, indob_scale5] /= 1e6
        sln_bar[:, indob_scale6] /= 1e7
        sln_bar[:, indob_scale7] /= 1e8
        sln_bar[:, indob_scale8] /= 1e9
        sln_bar[:, indob_scale9] /= 1e10
        sln_bar[:, indob_scale10] /= 1e12
        sln_bar[:, indob_scale11] /= 1e13
        sln_bar[:, indob_scale12] /= 1e14
        sln_bar[:, indob_scale13] /= 1e15
        sln_bar[:, indob_scale14] /= 1e16
        sln_bar[:, indob_scale15] /= 1e17

        x0_EnSF = sln_bar.copy()
    

    x_state[:, idx_obs] = sln_bar.copy()
    
    # ## U
    Uens = x_state[:, :Size_U].copy()
    img_maskU = np.zeros((Ny, Nx))
   
    img_maskU[idx_Uobs_rows, idx_Uobs_cols] = 1.0
    masked_imgU = Uens.reshape(ensemble_size, Ny, Nx)
    
    masked_img_minU = np.min(masked_imgU, axis = (1,2))[:, None, None] #
    masked_img_maxU = np.max(masked_imgU, axis = (1,2))[:, None, None]  #
    masked_imgU = (masked_imgU - masked_img_minU)/ (masked_img_maxU - masked_img_minU)
    masked_imgU = masked_imgU * img_maskU
    recovered_imgU = np.zeros((ensemble_size,Ny,Nx))
    
    num_imagesU = ensemble_size
    results_imgU =\
        Parallel(n_jobs=-1)(delayed(inpaint_task)(jjjj, masked_imgU, img_maskU) for jjjj in range(num_imagesU))
    
    recovered_imgU[:] = np.array(results_imgU)
    recovered_imgU = recovered_imgU * (masked_img_maxU - masked_img_minU) + masked_img_minU

    Uens[:, idxU_unobs] = recovered_imgU.reshape(ensemble_size,Size_U)[:, idxU_unobs] 
    
    ## V
    Vens = x_state[:, Size_U+np.arange(0, Size_V)].copy()
    img_maskV = np.zeros((Ny, Nx))
   
    img_maskV[idx_Vobs_rows, idx_Vobs_cols] = 1.0
    masked_imgV = Vens.reshape(ensemble_size, Ny, Nx)
    
    masked_img_minV = np.min(masked_imgV, axis = (1,2))[:, None, None] #
    masked_img_maxV = np.max(masked_imgV, axis = (1,2))[:, None, None]  #
    masked_imgV = (masked_imgV - masked_img_minV)/ (masked_img_maxV - masked_img_minV)
    masked_imgV = masked_imgV * img_maskV
    recovered_imgV = np.zeros((ensemble_size,Ny, Nx))
    
    num_imagesV = ensemble_size
    results_imgV =\
        Parallel(n_jobs=-1)(delayed(inpaint_task)(jjjj, masked_imgV, img_maskV) for jjjj in range(num_imagesV))
    
    recovered_imgV[:] = np.array(results_imgV)
    recovered_imgV = recovered_imgV * (masked_img_maxV - masked_img_minV) + masked_img_minV

    Vens[:, idxV_unobs-Size_U] = recovered_imgV.reshape(ensemble_size, Size_V)[:, idxV_unobs-Size_U] 
    
    
    ## P
    Pens = x_state[:, Size_U+Size_V+np.arange(0, Size_P)].copy()
    img_maskP = np.zeros((Ny, Nx))
   
    img_maskP[idx_Pobs_rows, idx_Pobs_cols] = 1.0
    masked_imgP = Pens.reshape(ensemble_size, Ny, Nx)
    
    masked_img_minP = np.min(masked_imgP, axis = (1,2))[:, None, None] #
    masked_img_maxP = np.max(masked_imgP, axis = (1,2))[:, None, None]  #
    masked_imgP = (masked_imgP - masked_img_minP)/ (masked_img_maxP - masked_img_minP)
    masked_imgP = masked_imgP * img_maskP
    recovered_imgP = np.zeros((ensemble_size, Ny, Nx))
    
    num_imagesP = ensemble_size
    results_imgP =\
        Parallel(n_jobs=-1)(delayed(inpaint_task)(jjjj, masked_imgP, img_maskP) for jjjj in range(num_imagesP))
    
    recovered_imgP[:] = np.array(results_imgP)
    recovered_imgP = recovered_imgP*(masked_img_maxP-masked_img_minP)+masked_img_minP

    Pens[:, idxP_unobs-Size_U-Size_V] = recovered_imgP.reshape(ensemble_size,Size_P)[:, idxP_unobs-Size_U-Size_V] 
    
    #### Update unobserved index
    state_from_image = np.concatenate((Uens, Vens, Pens), axis=1)
    
    obs_unindx = state_from_image[:, idx_unobs].copy()

    sln_unobs = reverse_SDE(obs_unindx, obs_sigma, x0=x_state[:, idx_unobs].copy(),\
                            time_steps=euler_steps,  C=1, score_likelihood_EnSF=score_likelihood_unbos)
    
    x_state[:, idx_unobs] = sln_unobs.copy()
    
    ## Setting for the next time step
    x_state[:, Size_U+Size_V+np.arange(0, Size_P)] = np.clip(x_state[:, Size_U+Size_V+np.arange(0, Size_P)], -0.55, 0.55)
    x_state[:, -1] = np.clip(x_state[:, -1], 0.8, 1.05)
    
    # print(x_state[:, Size_U+Size_V+np.arange(0, Size_P)])
    U0_state = x_state[:, :Size_U].copy()
    V0_state = x_state[:, Size_U+np.arange(0, Size_V)].copy()
    q_batch = x_state[:, -1].copy()
    print(q_Ref[i+1])
    # print(q_batch)
    est_save[[i+1], :] += np.mean(x_state, axis=0)
    print(est_save[i+1, -1])
    # get rmse
    rmse_temp = np.sqrt(np.mean((est_save[[i+1], :] - state_timeextract[[i+1], :])**2))

    # get time
    t2 = time.time()
    print(f'\t RMSE = {rmse_temp:.4f}')
    print(f'\t time = {t2 - t1:.4f}')

    # save information
    rmse_all.append(rmse_temp)

    # check divergence
    if rmse_temp > 1000:
        print('diverge!')
        break

step=0:
[1.]
1.0177969487074439
	 RMSE = 0.4265
	 time = 89.7646
step=1:
[1.]
0.963756754780755
	 RMSE = 0.3929
	 time = 85.8532
step=2:
[1.]
0.9387494554324546
	 RMSE = 0.3645
	 time = 85.7119
step=3:
[1.]
0.9192316750210824
	 RMSE = 0.3372
	 time = 74.7982
step=4:
[1.]
0.8923071718971088
	 RMSE = 0.3081
	 time = 76.2477
step=5:
[1.]
0.9395388518716475
	 RMSE = 0.2831
	 time = 76.1855
step=6:
[1.]
0.9707509510086004
	 RMSE = 0.2630
	 time = 84.0643
step=7:
[1.]
0.9413734395259402
	 RMSE = 0.2459
	 time = 90.4653
step=8:
[1.]
0.956398223369211
	 RMSE = 0.2272
	 time = 80.6456
step=9:
[1.]
0.9438821768437817
	 RMSE = 0.2138
	 time = 84.1283
step=10:
[1.]
0.9388774225869954
	 RMSE = 0.1980
	 time = 81.0151
step=11:
[1.]
0.9595693467115644
	 RMSE = 0.1855
	 time = 82.1118
step=12:
[1.]
0.9836789325693749
	 RMSE = 0.1738
	 time = 79.3543
step=13:
[1.]
0.9939542752927535
	 RMSE = 0.1586
	 time = 81.7296
step=14:
[1.]
0.953456340269652
	 RMSE = 0.1515
	 time = 81.3507
step=15:
[1.]
0.9751454

In [95]:
U_EnSF = est_save[:, :Size_U]
V_EnSF = est_save[:, Size_U+np.arange(0, Size_V)]
P_EnSF = est_save[:, Size_U+Size_V+np.arange(0, Size_P)]
q_EnSF = est_save[:, -1]

In [97]:
scipy.io.savemat('ResultEnSF_Periodic_T100_7Obs_noise0001_BiH_v1.mat', {'U_EnSF':U_EnSF, 'V_EnSF':V_EnSF, 'P_EnSF':P_EnSF, \
                                                                        'q_EnSF': q_EnSF, 'rmse': rmse_all})

# scipy.io.savemat('ResultEnSF_Periodic_T100_7Obs_noise01_BiH_v1.mat', {'U_EnSF':U_EnSF, 'V_EnSF':V_EnSF, 'P_EnSF':P_EnSF, \
#                                                                       'q_EnSF': q_EnSF, 'rmse': rmse_all})